In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing libaries
import pandas as pd
import numpy as np

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Rescaling all the images 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_normalize = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_normalize = ImageDataGenerator(rescale=1./255)
val_normalize = ImageDataGenerator(rescale=1./255)

In [ ]:
# load the images from their own directories
train_set = train_normalize.flow_from_directory(directory="/kaggle/input/plant-disease-recognition-dataset/Train/Train",target_size=(225,225),class_mode='categorical')
val_set = val_normalize.flow_from_directory(directory="/kaggle/input/plant-disease-recognition-dataset/Validation/Validation",target_size=(225,225),class_mode="categorical")
test_set = test_normalize.flow_from_directory(directory="/kaggle/input/plant-disease-recognition-dataset/Test/Test",target_size=(225,225),class_mode="categorical")

In [ ]:
# imporing necessary libaries
from tensorflow.keras import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D,Dropout

In [ ]:
# CNN model architecture
model = Sequential()
model.add(Conv2D(64,(3,3),input_shape=(225,225,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))

In [ ]:
# plot the model
plot_model(model,show_layer_names=True,show_shapes=True)

In [ ]:
# Using categorical loss function using adam optimizer
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# fit the dataset in to the model
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_set, epochs=9, batch_size=16, validation_data=test_set, callbacks=[early_stopping])

In [ ]:
# plot the performance
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
model.evaluate(val_set)

In [ ]:
model.evaluate(test_set)

In [ ]:
# Prediction
from tensorflow.keras.preprocessing.image import img_to_array,load_img

classes =train_set.class_indices
icd={i:j for j,i in classes.items()} 
def output(location):
    img=load_img(location,target_size=(225,225,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    predict_x=model.predict(img) 
    classes_x=np.argmax(predict_x,axis=1)
    print ('Pridicted Class',icd[classes_x[0]])

In [ ]:
test_img='/kaggle/input/plant-disease-recognition-dataset/Test/Test/Rust/830f26cc6d3cd13e.jpg'
pic=load_img(test_img,target_size=(225,225,3))
plt.imshow(pic)
output(test_img)

In [ ]:
test_img='/kaggle/input/plant-disease-recognition-dataset/Test/Test/Powdery/81c36e378473c93e.jpg'
pic=load_img(test_img,target_size=(225,225,3))
plt.imshow(pic)
output(test_img)

In [ ]:
test_img='/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Healthy/9d6acb154f9932c8.jpg'
pic=load_img(test_img,target_size=(225,225,3))
plt.imshow(pic)
output(test_img)

In [ ]:
test_img='/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Powdery/9b6a318cc5721d73.jpg'
pic=load_img(test_img,target_size=(225,225,3))
plt.imshow(pic)
output(test_img)